<a href="https://colab.research.google.com/github/ankita1001/hello-world/blob/master/Test/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
#Clone the retina net git repositary
!git clone https://github.com/fizyr/keras-retinanet.git
%cd keras-retinanet/
!pip install .
!python setup.py build_ext --inplace

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 5893 (delta 3), reused 0 (delta 0), pack-reused 5885
Receiving objects: 100% (5893/5893), 14.81 MiB | 35.94 MiB/s, done.
Resolving deltas: 100% (3962/3962), done.
/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170852 sha256=58a176616e331e721b054aee10a29afeac68a125db8c307e5f4cf3bb135670a2
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=3e27329b7f712f69342b1a1e6d1c27b87c31fa7d42a30e0e80c4ad73278a3973
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet kera

In [2]:
import os
os.getcwd()

'/content'

In [5]:
import os
import sys
import math
import glob
import argparse
import xml.etree.ElementTree as ET

In [9]:
train_data = '/content/drive/My Drive/Object Detection/Train Data/'
test_data = '/content/drive/My Drive/Object Detection/Test Data/'

In [10]:

train_annotations = './annotations.csv'
val_annotations = './val_annotations.csv'
label_file = './classes.csv'
# CONVERT the XML annotations to CSV format
def convert_annotation(train_data, test_data, image_id, filename, classes, train=False):
    if train:
        in_file = open(train_data + '%s.xml' % image_id)
    else:
        in_file = open(test_data + '%s.xml' % image_id)
    out_file = open(filename, 'a')
    tree = ET.parse(in_file)
    root = tree.getroot()

    if root.iter('object') is not None:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)

            xmlbox = obj.find('bndbox')
            x1 = math.ceil(float(xmlbox.find('xmin').text))
            y1 = math.ceil(float(xmlbox.find('ymin').text))
            x2 = math.ceil(float(xmlbox.find('xmax').text))
            y2 = math.ceil(float(xmlbox.find('ymax').text))
            if x1 == x2 or y1 == y2:
                continue
            if train:
                out_file.write(
                    f'{train_data + image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
            else:
                out_file.write(
                    f'{test_data + image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
    else:
        if train:
            out_file.write(f'{train_data + image_id}.jpg,,,,,\n')
        else:
            out_file.write(f'{test_data + image_id}.jpg,,,,,\n')







    # filter out the train test jpg files
train_image = glob.glob(train_data + '/*.jpg')
test_image = glob.glob(test_data + '/*.jpg')

    # define the classes for the model
labels = ['chair','table','vase']

    # create new annotation files for  Train/validation
open(train_annotations, 'w')
open(val_annotations, 'w')

train_ids = [os.path.basename(i[:-4]) for i in train_image]
for image_id in train_ids:
    convert_annotation(train_data, test_data, image_id, train_annotations, labels, train=True)
val_ids = [os.path.basename(i[:-4]) for i in test_image]

for image_id in val_ids:
    convert_annotation(train_data, test_data, image_id, val_annotations, labels)

    # creating classes file listing all classes
with open(label_file, 'w') as f:
    for i, line in enumerate(labels):
        f.write('{},{}\n'.format(line, i))

print('DONE')






DONE


In [11]:
os.makedirs("snapshots",exist_ok=True)
os.makedirs("logs",exist_ok=True)

In [13]:
from six.moves import urllib

In [14]:

    

#Store the pretrained model downloaded from the url in the snapshot folder
pretrained_model = ("./snapshots/_pretrained_model.h5")
url = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(url,pretrained_model)




('./snapshots/_pretrained_model.h5',
 <http.client.HTTPMessage at 0x7f995ff6c9e8>)

In [15]:

    

!./keras_retinanet/bin/train.py --backbone='resnet50' --weights {pretrained_model} --random-transform --batch-size=4 --steps 800 --epochs 10 --image-min-side 500 --image-max-side 833 --lr 0.001 --weighted-average csv annotations.csv classes.csv --val-annotations val_annotations.csv



Using TensorFlow backend.
2020-07-30 11:14:50.392026: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Creating model, this may take a second...
2020-07-30 11:15:59.778536: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-30 11:15:59.848171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-30 11:15:59.848822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-07-30 11:15:59.848866: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-

Inference



In [31]:
import keras


In [32]:
# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu


In [33]:
# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

In [34]:
model_path = os.path.join('/content/keras-retinanet', 'snapshots', 'resnet50_csv_06.h5')
# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
# if the model is not converted to an inference model, use the line below

#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'chair', 1: 'table', 2: 'vase'}

In [35]:
print(model.summary())

Model: "retinanet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________

In [44]:
image = read_image_bgr('/content/drive/My Drive/Object Detection/detection5.jpg')

In [45]:
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)


In [46]:
# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

In [47]:
# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

ValueError: ignored